# Measuring cuboids


In [ ]:
# You can only fill in a file with a .ply extension
# see data.ipynb to use the data from the dataset
fileName = 'pointcloud1'

# Coordinates of point in the top plane of the cuboid
top_plane_point_x = -0.060090084
top_plane_point_y = -0.15732084
top_plane_point_z = -0.2964331

## 1. Importing libraries and data

In [ ]:
import open3d as o3d

# remove outliers
cloud = o3d.io.read_point_cloud(fileName + ".ply")

## 2. Statistical Outlier Filter

In [ ]:
cloud, ind = cloud.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
o3d.io.write_point_cloud(f"{fileName}_1_statOutlierRemoval.ply", cloud)

## 3. Radius Outlier Removal

In [ ]:
cloud, ind = cloud.remove_radius_outlier(nb_points=10, radius=0.01)
o3d.io.write_point_cloud(f"{fileName}_2_removeRadiusOutlier.ply", cloud)

## 4. Voxel Down Sample

In [ ]:
cloud = cloud.voxel_down_sample(voxel_size=0.005)
o3d.io.write_point_cloud(f"{fileName}_3_voxelDownSample.ply", cloud)

# 5. Floor removal

In [ ]:
import numpy as np

normal = np.array([0, 1, 0])
top_plane_point = np.array([top_plane_point_x, top_plane_point_y, top_plane_point_z])
top_plane_point_product = -np.dot(normal, top_plane_point)
points = np.asarray(cloud.points)

threshold = 0.02

points_not_in_plane = points[~((np.abs(np.dot(normal, points.T) + top_plane_point_product)) < threshold)]


In [ ]:
# Visualize
v_points_in_plane = np.arange(points.shape[0])[(np.abs(np.dot(normal, points.T) + top_plane_point_product)) < threshold]
v_points_not_in_plane = np.arange(points.shape[0])[~((np.abs(np.dot(normal, points.T) + top_plane_point_product)) < threshold)]


visual = cloud.select_by_index(v_points_not_in_plane, invert=False)
visual.paint_uniform_color([0, 1, 0])
visual2 = cloud.select_by_index(v_points_in_plane, invert=False)
visual2.paint_uniform_color([0, 0, 1])
o3d.io.write_point_cloud(f"{fileName}_x_pointsNotIP.ply", visual)
o3d.io.write_point_cloud(f"{fileName}_x_pointsIP.ply", visual2)
o3d.io.write_point_cloud(f"{fileName}_x_combined.ply", visual + visual2)

# draw tap point
# mesh = o3d.geometry.TriangleMesh.create_sphere(radius=0.02)
# mesh.translate(top_plane_point)

# o3d.visualization.draw_geometries([mesh, cloud])

In [ ]:
import pyransac3d as pyrsc
# convert cloud to numpy array
floor_plane = pyrsc.Plane()
floor_cutting = 0.02

best_eq, best_inliers = floor_plane.fit(points_not_in_plane, floor_cutting, 1000)

# Points that do not belong to the floor found floorplane
points_no_floor = points[~((np.abs(np.dot(best_eq[0:3], points.T) + best_eq[3])) < threshold)]
# Points that do not belong to the floor found floorplane and are not below the floor
points_not_below_floor = points_no_floor[~(points_no_floor[:, 1] < (-best_eq[3] / best_eq[1]))] 

newCloud = o3d.geometry.PointCloud()
newCloud.points = o3d.utility.Vector3dVector(np.array(points_not_below_floor))
# o3d.visualization.draw_geometries([newCloud])
o3d.io.write_point_cloud(f"{fileName}_4_floorPlane.ply", newCloud)

<!-- \texttt{floor\_plane.get\_oriented\_bounding\_box():} Deze methode berekent de georiënteerde bounding box die past rond de punten die behoren tot \texttt{floor\_plane} object en slaat het resultaat op in het obb object. \texttt{obb.color = [0, 0, 1]} zet de kleur van de boundingbox op blauw. \\

\texttt{obb.get\_point\_indices\_within\_bounding\_box(cloud.points)} geeft een lijst van indices van punten in de puntewolk terug die zich binnen de bounding box bevinden. -->

## 6. Isolating the cuboid

In [ ]:
# Alle punten vinden die in de vorm van een kubus zijn
cuboid = pyrsc.Cuboid()
# best_eq is een numpy matrix van 3 beste vlakkenvergelijking
# best_inliers is een lijst van de indexen van de punten die in de kubus passen
best_eq, best_inliers = cuboid.fit(np.asarray(points_not_below_floor), thresh=0.005)

# VISUALISATIE START
# cuboid_pcl is een pointcloud van de punten die in de kubus passen
# not_cuboid_pcl is een pointcloud van de punten die niet in de kubus passen
cuboid_pcl = newCloud.select_by_index(best_inliers).paint_uniform_color([1, 0, 0])
not_cuboid_pcl = newCloud.select_by_index(best_inliers, invert=True).paint_uniform_color([0, 1, 0])
plane = newCloud.select_by_index(best_inliers).paint_uniform_color([1, 0, 0])

# Join  plane and not plane togheter 
o3d.io.write_point_cloud(f"{fileName}_6_cuboidPcl.ply", cuboid_pcl)
o3d.io.write_point_cloud(f"{fileName}_7_nonCuboidPcl.ply", not_cuboid_pcl)
o3d.io.write_point_cloud(f"{fileName}_8_combinedCuboidAndNonCuboidPcl.ply", cuboid_pcl + not_cuboid_pcl)
# VISUALISATIE EINDE

In [ ]:
horizontal_plane_idx = 0
left_plane_idx = 1
right_plane_idx = 2

# Haal de beste vlakkenvergelijkingen op voor de horizontale, linker en rechter vlakken
top_plane = best_eq[horizontal_plane_idx]
left_plane = best_eq[left_plane_idx]
right_plane = best_eq[right_plane_idx]

# VISUALISATIE START

# VISUALISATIE EINDE

## 7. Calculating vectors



In [ ]:
def plane_intersect(a, b):
    """
    a, b   4-tuples/lists
           Ax + By +Cz + D = 0
           A,B,C,D in order  

    output is een punt en een vector

p_inter is punt op d eintersectielijn en aXb_vec is de richtingsvector
    """
    a_vec, b_vec = np.array(a[:3]), np.array(b[:3])

    aXb_vec = np.cross(a_vec, b_vec)

    A = np.array([a_vec, b_vec, aXb_vec])
    d = np.array([-a[3], -b[3], 0.]).reshape(3,1)

    p_inter = np.linalg.solve(A, d).T

    return p_inter[0], aXb_vec

In [ ]:
_, new_y_vector = plane_intersect(left_plane, right_plane)
_, new_x_vector = plane_intersect(top_plane, right_plane)
new_z_vector = np.cross(new_x_vector, new_y_vector)

In [ ]:
# normalize vectors
new_x_vector = new_x_vector / np.linalg.norm(new_x_vector)
new_y_vector = new_y_vector / np.linalg.norm(new_y_vector)
new_z_vector = new_z_vector / np.linalg.norm(new_z_vector)

# 8. Rotating the cuboid

In [ ]:
cuboid_pcl_points = np.asarray(cuboid_pcl.points)

base_change_matrix = np.array([new_x_vector, new_y_vector, new_z_vector]).reshape(3,3)
new_points = np.dot(base_change_matrix, cuboid_pcl_points.T).T

In [ ]:
# to pointcloud
new_cuboid_pcl = o3d.geometry.PointCloud()
new_cuboid_pcl.points = o3d.utility.Vector3dVector(new_points)


In [ ]:
def selectClosestCluster(pcd, top_plane_point):
    pcdPoints = np.asarray(pcd.points)
    clusterLabels = np.asarray(pcd.cluster_dbscan(eps=0.01, min_points=10))

    nonNoisePoints = pcdPoints[clusterLabels != -1]
    nonNoiseClusterLabels = clusterLabels[clusterLabels != -1]
    pointDistances = np.linalg.norm(nonNoisePoints - np.asarray(top_plane_point), axis=1)

    closestPointIdx = np.argmin(pointDistances)
    closestPointLabel = nonNoiseClusterLabels[closestPointIdx]

    new_cuboid_pcl = o3d.geometry.PointCloud()
    new_cuboid_pcl.points = o3d.utility.Vector3dVector(pcdPoints[clusterLabels == closestPointLabel])

    return new_cuboid_pcl

new_cuboid_pcl = selectClosestCluster(new_cuboid_pcl, top_plane_point)

# 9. Fitting the bounding box and returing the measurements


In [ ]:
# get axis aligned bounding box
bb = new_cuboid_pcl.get_axis_aligned_bounding_box()
bb.color = [0, 1, 0]

In [ ]:
# translate bounding box to original coordinate system
bb_points = np.asarray(bb.get_box_points())
bb_points = np.dot(np.linalg.inv(base_change_matrix), bb_points.T).T

In [ ]:

# Visualize the oriented bounding box
# Create an oriented bounding box from the transformed points
new_bb = o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(bb_points))
new_bb.color = [0, 1, 0]


width = bb.max_bound[0] - bb.min_bound[0]
height = bb.max_bound[1] - bb.min_bound[1] + 0.02
depth = bb.max_bound[2] - bb.min_bound[2]

print(f"Width: {width}, Height: {height}, Depth: {depth}")


In [ ]:
o3d.visualization.draw_geometries([cloud, new_bb])